In [5]:
# Setup
import os
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient

# Load configuration
load_dotenv('config.env')
base_url = os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321')
model_id = os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16')

print(f"🔗 Connecting to: {base_url}")
print(f"🤖 Using model: {model_id}")

# Create client
client = LlamaStackClient(base_url=base_url)
print("✅ Client created")

# Check available MCP tools
print("\n🔍 Checking available MCP Atlassian tools...")
try:
    tools = client.tools.list(toolgroup_id="mcp::atlassian")
    print(f"✅ Found {len(tools)} Atlassian tools")
    
    # Find Jira tools
    jira_tools = [t for t in tools if 'jira' in str(t).lower()]
    print(f"🎫 Jira tools: {len(jira_tools)}")
    
    # Show create and search tools
    for tool in jira_tools:
        tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
        if 'create' in tool_name or 'search' in tool_name:
            print(f"  - {tool_name}")
            
except Exception as e:
    print(f"❌ Error loading tools: {e}")


🔗 Connecting to: http://localhost:8321
🤖 Using model: r1-qwen-14b-w4a16
✅ Client created

🔍 Checking available MCP Atlassian tools...
✅ Found 37 Atlassian tools
🎫 Jira tools: 27
  - jira_batch_create_issues
  - jira_create_issue
  - jira_create_issue_link
  - jira_create_sprint
  - jira_search
  - jira_search_fields


In [6]:
# Test 1: Simple Jira Issue Creation using Direct Tool Runtime
print("🧪 Test 1: Create Simple Jira Issue using Direct Tool Runtime")
print("="*65)

try:
    # Use direct tool invocation to create Jira issue
    result1 = client.tool_runtime.invoke_tool(
        tool_name="jira_create_issue",
        kwargs={
            "project_key": "KAN",
            "summary": "Direct Tool Runtime Test Issue",
            "issue_type": "Task",
            "description": "This issue was created using direct tool runtime API"
        }
    )
    
    print("✅ Jira issue created successfully!")
    print(f"Issue result: {result1}")
    
    # Extract issue key if available
    if hasattr(result1, 'key'):
        issue_key = result1.key
        print(f"🎫 Created issue: {issue_key}")
    else:
        print("📝 Issue created (key not available in response)")
    
    print("\n✅ Test 1 completed!")
    
except Exception as e:
    print(f"❌ Test 1 failed: {e}")
    import traceback
    traceback.print_exc()


🧪 Test 1: Create Simple Jira Issue with Automatic MCP Tool Calling
❌ Test 1 failed: Error code: 400 - {'error': {'detail': {'errors': [{'loc': ['body', 'tools', 0], 'msg': "Input tag 'mcp' found using 'type' does not match any of the expected tags: 'web_search', 'web_search_preview_2025_03_11', 'file_search', 'function'", 'type': 'union_tag_invalid'}]}}}


Traceback (most recent call last):
  File "/var/folders/9w/xmlk078553sb8381sw9hh5dc0000gn/T/ipykernel_33495/3254295698.py", line 7, in <module>
    response1 = client.responses.create(
        model=model_id,
    ...<6 lines>...
        ]
    )
  File "/Users/sudash/Desktop/MyFiles/Sukanta/AIProjects/citi/mcp-test-env/lib/python3.13/site-packages/llama_stack_client/_utils/_utils.py", line 283, in wrapper
    return func(*args, **kwargs)
  File "/Users/sudash/Desktop/MyFiles/Sukanta/AIProjects/citi/mcp-test-env/lib/python3.13/site-packages/llama_stack_client/resources/responses.py", line 183, in create
    return self._post(
           ~~~~~~~~~~^
        "/v1/openai/v1/responses",
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<18 lines>...
        stream_cls=Stream[ResponseObjectStream],
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/sudash/Desktop/MyFiles/Sukanta/AIProjects/citi/mcp-test-env/lib/python3.13/site-packages/llama_stack_client/_base_client.py", lin

In [ ]:
# Test 2: OOM Error Incident Creation using Direct Tool Runtime
print("\n🧪 Test 2: Create OOM Error Incident using Direct Tool Runtime")
print("="*65)

try:
    # Use direct tool invocation to create OOM incident
    result2 = client.tool_runtime.invoke_tool(
        tool_name="jira_create_issue",
        kwargs={
            "project_key": "KAN",
            "summary": "Pod failing due to OOM error",
            "issue_type": "Incident",
            "description": "Pod experiencing Out of Memory errors causing application failures",
            "additional_fields": {
                "priority": {"name": "High"},
                "labels": ["oom-error", "pod-failure", "high-priority"]
            }
        }
    )
    
    print("✅ OOM incident created successfully!")
    print(f"Incident result: {result2}")
    
    # Extract issue key if available
    if hasattr(result2, 'key'):
        incident_key = result2.key
        print(f"🚨 Created incident: {incident_key}")
    else:
        print("📝 Incident created (key not available in response)")
    
    print("\n✅ Test 2 completed!")
    
except Exception as e:
    print(f"❌ Test 2 failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: Search for Created Issues using Direct Tool Runtime
print("\n🧪 Test 3: Search for Created Issues using Direct Tool Runtime")
print("="*65)

try:
    # Use direct tool invocation to search for issues
    result3 = client.tool_runtime.invoke_tool(
        tool_name="jira_search",
        kwargs={
            "jql": "project = KAN ORDER BY created DESC"
        }
    )
    
    print("✅ Search completed successfully!")
    print(f"Search result: {result3}")
    
    # Check if our test issues were created
    search_text = str(result3)
    if "Direct Tool Runtime Test Issue" in search_text:
        print("🎉 SUCCESS: Direct Tool Runtime Test Issue found!")
    else:
        print("⚠️  Direct Tool Runtime Test Issue not found")
        
    if "OOM error" in search_text:
        print("🎉 SUCCESS: OOM error incident found!")
    else:
        print("⚠️  OOM error incident not found")
    
    print("\n✅ Test 3 completed!")
    
except Exception as e:
    print(f"❌ Test 3 failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 4: Additional Tool Testing (Get Issue Details)
print("\n🧪 Test 4: Additional Tool Testing (Get Issue Details)")
print("="*60)

try:
    # First, let's search for issues to get a specific issue key
    search_result = client.tool_runtime.invoke_tool(
        tool_name="jira_search",
        kwargs={"jql": "project = KAN ORDER BY created DESC"}
    )
    
    print("✅ Search completed!")
    print(f"Search result: {search_result}")
    
    # Try to get project information
    project_info = client.tool_runtime.invoke_tool(
        tool_name="jira_get_project_issues",
        kwargs={"project_key": "KAN"}
    )
    
    print("✅ Project info retrieved!")
    print(f"Project issues: {project_info}")
        
except Exception as e:
    print(f"❌ Additional testing failed: {e}")
    import traceback
    traceback.print_exc()
